# Electric Vehicle Population Data for Washington State

### Current registered EVs in the State of Washington according to Washington State Department of Licensing (DOL)


## <ins>Introduction: Cleaning and exploring the dataset</ins>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
import hvplot.pandas 
import holoviews as hv
from bokeh.models import LogTicker, FuncTickFormatter, FixedTicker

In [2]:
df = pd.read_csv('./csv/Electric_Vehicle_Population_Data.csv')

print(len(df.columns))
print(df.info())
df.head(1)

17
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247344 entries, 0 to 247343
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         247344 non-null  object 
 1   County                                             247340 non-null  object 
 2   City                                               247340 non-null  object 
 3   State                                              247344 non-null  object 
 4   Postal Code                                        247340 non-null  float64
 5   Model Year                                         247344 non-null  int64  
 6   Make                                               247344 non-null  object 
 7   Model                                              247344 non-null  object 
 8   Electric Vehicle Type                              247344 non-null  obj

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,1N4BZ0CP5G,King,Seattle,WA,98125.0,2016,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,46.0,349455557,POINT (-122.30253 47.72656),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10


### Checking various columns to see how many unique values there are
some of these columns Ill probably be able to convert to categories instead of strings/objects for better performance
Also based on the info method call above, and looking at column types, I might convert other column types where it makes sense

As an example, some of the float columns would make more sense if they are ints, i.e. postal code

In [3]:
df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].unique()


array(['Clean Alternative Fuel Vehicle Eligible',
       'Not eligible due to low battery range',
       'Eligibility unknown as battery range has not been researched'],
      dtype=object)

In [4]:
df['Electric Vehicle Type'].unique()

array(['Battery Electric Vehicle (BEV)',
       'Plug-in Hybrid Electric Vehicle (PHEV)'], dtype=object)

In [5]:
df['Make'].unique()

array(['NISSAN', 'KIA', 'TESLA', 'JEEP', 'FORD', 'FIAT', 'AUDI',
       'CHEVROLET', 'PORSCHE', 'HYUNDAI', 'BMW', 'RIVIAN', 'MAZDA',
       'POLESTAR', 'TOYOTA', 'VOLVO', 'SUBARU', 'LEXUS', 'MINI', 'HONDA',
       'MERCEDES-BENZ', 'CHRYSLER', 'VOLKSWAGEN', 'ALFA ROMEO', 'SMART',
       'JAGUAR', 'CADILLAC', 'LINCOLN', 'MITSUBISHI', 'DODGE',
       'LAND ROVER', 'ACURA', 'GMC', 'GENESIS', 'LUCID', 'FISKER',
       'VINFAST', 'LAMBORGHINI', 'TH!NK', 'MULLEN AUTOMOTIVE INC.',
       'ROLLS-ROYCE', 'BENTLEY', 'BRIGHTDROP', 'AZURE DYNAMICS', 'RAM',
       'WHEEGO ELECTRIC CARS'], dtype=object)

In [6]:
df['State'].unique()

array(['WA', 'BC', 'CA', 'NY', 'OR', 'WY', 'CO', 'MD', 'FL', 'NC', 'MN',
       'VA', 'AZ', 'IL', 'NJ', 'GU', 'TX', 'NE', 'MO', 'IN', 'MA', 'AP',
       'MI', 'GA', 'ID', 'UT', 'DC', 'SC', 'CT', 'AR', 'RI', 'NV', 'PA',
       'KS', 'LA', 'OK', 'AL', 'HI', 'KY', 'NH', 'OH', 'NM', 'TN', 'ME',
       'WI', 'DE', 'NS', 'MS', 'AK'], dtype=object)

In [7]:
df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].unique()

array(['Clean Alternative Fuel Vehicle Eligible',
       'Not eligible due to low battery range',
       'Eligibility unknown as battery range has not been researched'],
      dtype=object)

Converting columns

In [8]:
df['State'] = df['State'].astype('category')
df['Electric Vehicle Type'] = df['Electric Vehicle Type'].astype('category')
df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] = df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].astype('category')
df['Postal Code'] = df['Postal Code'].astype('Int64')
df['Electric Range'] = df['Electric Range'].astype('Int64')
df['Legislative District'] = df['Legislative District'].astype('Int64')



Adding extra region column for comparison later

In [9]:
north_america = ['TESLA', 'JEEP', 'FORD', 'CHEVROLET', 'RIVIAN', 'CHRYSLER', 'CADILLAC', 'LINCOLN', 'DODGE', 'GMC', 'LUCID', 'FISKER', 'MULLEN AUTOMOTIVE INC.', 'BRIGHTDROP', 'RAM', 'AZURE DYNAMICS', 'WHEEGO ELECTRIC CARS']
asia = ['NISSAN', 'KIA', 'HYUNDAI', 'MAZDA', 'TOYOTA', 'SUBARU', 'LEXUS', 'HONDA', 'MITSUBISHI', 'ACURA', 'GENESIS', 'VINFAST']
europe = ['FIAT', 'AUDI', 'PORSCHE', 'BMW', 'POLESTAR', 'VOLVO', 'MINI', 'MERCEDES-BENZ', 'VOLKSWAGEN', 'ALFA ROMEO', 'SMART', 'JAGUAR', 'LAND ROVER', 'LAMBORGHINI', 'TH!NK', 'ROLLS-ROYCE', 'BENTLEY']
def region_check(car_make):
    if car_make in north_america:
        return 'North America'
    elif car_make in asia:
        return 'Asia'
    else:
        return 'Europe'


df['EV Regional Origin'] = df['Make'].apply(region_check)

df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract,EV Regional Origin
0,1N4BZ0CP5G,King,Seattle,WA,98125,2016,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84,0.0,46,349455557,POINT (-122.30253 47.72656),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10,Asia
1,KNDJX3AEXG,King,Renton,WA,98058,2016,KIA,SOUL,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,93,31950.0,11,210641315,POINT (-122.08747 47.4466),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10,Asia
2,5YJ3E1EB2J,King,Seattle,WA,98115,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0.0,43,171417494,POINT (-122.31676 47.68156),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10,North America
3,1C4RJXN64R,Kitsap,Bremerton,WA,98312,2024,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21,0.0,26,262542927,POINT (-122.65223 47.57192),PUGET SOUND ENERGY INC,5.303508e+10,North America
4,5YJ3E1EB1J,Thurston,Olympia,WA,98512,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0.0,35,110432815,POINT (-122.9131 47.01359),PUGET SOUND ENERGY INC,5.306701e+10,North America


## Doing some exploration


### Bar Plot of number of EVs for Each Model Year

In [10]:
print(df['Make'].nunique())
print(df['Model Year'].unique())

filtered_df = df.groupby('Model Year').agg({'VIN (1-10)':'count'})
filtered_df.reset_index(inplace=True)
filtered_df.rename(columns={'VIN (1-10)': 'Number of EVs'}, inplace=True)
filtered_df.head()
hv_bar_plot = filtered_df.hvplot.bar(x='Model Year', 
                                    y='Number of EVs', 
                                    C='Count', 
                                    cmap='BuPu', 
                                    xlabel='Model Year', 
                                    ylabel='Number of Registered EVs (Log Scale)', 
                                    title='Number of Registered EVs in WA Per Model Year',
                                    logy=True).opts(
                                        show_grid=True
                                    )

textbox = hv.Text(
    x=2001, y=120,
    text='*Unit increase in Y = 10× more EVs'
).opts(
    text_align='left',
    text_font_size='10pt',
    bgcolor='white'
)

(hv_bar_plot * textbox)


46
[2016 2018 2024 2015 2019 2020 2012 2013 2022 2023 2017 2014 2021 2000
 2025 2011 2026 2010 2008 2002 2003]


:Overlay
   .Bars.I :Bars   [Model Year]   (Number of EVs)
   .Text.I :Text   [x,y]

## HeatMap of EVs in Washington State Counties

In [11]:
print(df[df['State'] == 'WA']['County'].nunique())

df_wa_counties = df[df['State'] == 'WA']

df_top_10_counties_ev_wa = df_wa_counties.groupby('County').agg({'VIN (1-10)':'count'}).nlargest(10, 'VIN (1-10)')
df_top_10_counties_ev_wa.reset_index(inplace=True)
# print(df_top_10_counties_ev_wa)

# print(df_wa_counties['Make'].nunique())

df_wa_counties = df_wa_counties[df_wa_counties['County'].isin(df_top_10_counties_ev_wa['County'])]

heat_map_data = df_wa_counties.pivot_table(index="County", columns="EV Regional Origin", values='VIN (1-10)', aggfunc='count', fill_value=0, sort=False)
# heat_map_data_norm = heat_map_data.apply(lambda x: x**2)
heat_map_data.head(20)


# heat_map_data.fillna(0)

# fig = plt.figure(figsize=(10,5))
# ax = fig.gca()
# sns.heatmap(heat_map_data, ax=ax, cmap='BuPu')

39


EV Regional Origin,Asia,North America,Europe
County,,,
King,24183,77790,20846
Kitsap,2295,4759,1321
Thurston,2455,5560,1066
Snohomish,6356,20595,3636
Skagit,717,1789,337
Spokane,1769,4319,883
Clark,3906,9216,1926
Pierce,4834,13118,2582
Whatcom,1923,3200,907


In [12]:
# heat_map_data.index.name = 'County'
# heat_map_data.columns.name = 'Region'

heat_map_long = heat_map_data.reset_index().melt(
    id_vars='County',
    var_name='EV Regional Origin',
    value_name='Number of EVs Registered'
)
print(heat_map_long.head()) 

custom_formatter = FuncTickFormatter(code="""
    var log = Math.log10(tick);
    return tick.toLocaleString() + " (log₁₀=" + log.toFixed(1) + ")";
""")

custom_ticks = [1000, 5000, 25000, 70000]

# Use FixedTicker instead of LogTicker
fixed_ticker = FixedTicker(ticks=custom_ticks)

hmap = heat_map_long.hvplot.heatmap(x='EV Regional Origin', 
                                    y='County', 
                                    C='Number of EVs Registered', 
                                    cmap='BuPu', 
                                    xlabel='EV Car Model Origin', 
                                    ylabel='Washington State County', 
                                    title='Top 10 WA Counties: EVs Registered by Car Origin (Log Scale)',
                                    logz=True,
                                    colorbar=True).opts(
                                        colorbar_opts={'title': 'Raw Number of EVs (w/ Log Scale)', 'ticker': fixed_ticker, 'formatter': custom_formatter}
                                        
                                    )
hmap

      County EV Regional Origin  Number of EVs Registered
0       King               Asia                     24183
1     Kitsap               Asia                      2295
2   Thurston               Asia                      2455
3  Snohomish               Asia                      6356
4     Skagit               Asia                       717


:HeatMap   [EV Regional Origin,County]   (Number of EVs Registered)

Violin Plot of Battery Range in each region

In [60]:
## for this one Ill look at all counties, not just in WA

new_df = df[df['EV Regional Origin'] == 'Europe']
state_df = df[df['State'] != 'WA']
print(new_df['Electric Range'].median())
print(df[df['EV Regional Origin'] == 'North America']['Electric Range'].isna().sum())
print(df['EV Regional Origin'].unique())


## remove any 0s for the electric range
violin__dot_df = df[df['Electric Range'] > 0]

# print(state_df)

# new_df.head(2)

12.0
0
['Asia' 'North America' 'Europe']


In [61]:
# plt.figure(figsize=(7, 7))
# sns.violinplot(
#     data=new_df,
#     x='EV Regional Origin',  # or y=... if you prefer horizontal
#     y='Electric Range',
#     palette=['red', 'silver', 'blue'],
#     hue="EV Regional Origin",
#     legend=True
# )
# plt.legend()
# plt.show()

violin = violin__dot_df.hvplot.violin(y='Electric Range', by='EV Regional Origin', ylabel='EV Range (Mi)',
                 cmap=['red', 'silver', 'blue'], legend=False, color='EV Regional Origin',
                 width=600, height=600, padding=0.4).opts(
                     ylim=(-25, 400)
                 )
violin

:Violin   [EV Regional Origin]   (Electric Range)

Swarm Plot of battery range v model year

In [85]:
## creating jitter
cat_codes = violin__dot_df['EV Regional Origin'].astype('category').cat.codes

violin__dot_df['Jitter'] = cat_codes + np.random.normal(0, 0.05, len(violin__dot_df))
# violin__dot_df.head()

scatter = violin__dot_df.hvplot.scatter(x='Model Year', y='Electric Range', by='Model Year',
                                        width=600, height=600, legend=False, hover_cols=[], tools=[])
scatter

/var/folders/_k/3wc9q0p9701fhmbcl77v_3h00000gq/T/ipykernel_96387/1500994238.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  violin__dot_df['Jitter'] = cat_codes + np.random.normal(0, 0.05, len(violin__dot_df))


:NdOverlay   [Model Year]
   :Scatter   [Model Year]   (Electric Range)